<a href="https://colab.research.google.com/github/Ravikamar/100-plus-Python-programming-exercises-extended/blob/master/SQL_Questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
try:
  from pyspark.sql import SparkSession
except:
  os.system("pip install pyspark")
from pyspark.sql import SparkSession

spark = SparkSession.builder.enableHiveSupport().getOrCreate()
from pyspark.sql.functions import *
import pyspark.sql.functions as F



```
Medium Level:
2️⃣
3️⃣
4️⃣ Write a query to calculate the running total of sales by date.
5️⃣ Find employees who earn more than the average salary in their department.
6️⃣ Write a query to find the most frequently occurring value in a column.
7️⃣ Fetch records where the date is within the last 7 days from today.
8️⃣ Write a query to count how many employees share the same salary.
9️⃣ How do you fetch the top 3 records for each group in a table?
🔟 Retrieve products that were never sold (hint: use LEFT JOIN).

💡 Challenging Level:
1️⃣ Retrieve customers who made their first purchase in the last 6 months.
2️⃣ How do you pivot a table to convert rows into columns?
3️⃣ Write a query to calculate the percentage change in sales month-over-month.
4️⃣ Find the median salary of employees in a table.
5️⃣ Fetch all users who logged in consecutively for 3 days or more.
6️⃣ Write a query to delete duplicate rows while keeping one occurrence.
7️⃣ Create a query to calculate the ratio of sales between two categories.
8️⃣ How would you implement a recursive query to generate a hierarchical structure?
9️⃣ Write a query to find gaps in sequential numbering within a table.
🔟 Split a comma-separated string into individual rows using SQL.

💡 Advanced Problem-Solving:
1️⃣ Rank products by sales in descending order for each region.
2️⃣ Fetch all employees whose salaries fall within the top 10% of their department.
3️⃣ Identify orders placed during business hours (e.g., 9 AM to 6 PM).
4️⃣ Write a query to get the count of users active on both weekdays and weekends.
5️⃣ Retrieve customers who made purchases across at least three different categories.
```



In [ ]:
print("1.M Write a query to find the second highest salary in an employee table.")

spark.sql("""SELECT * FROM VALUES
(1, 'Alice', 50000),
(2, 'Bob', 60000),
(3, 'Charlie', 70000),
(4, 'David', 60000)
AS t(id, name, salary)""").createOrReplaceTempView("tab")

spark.sql("select salary from (select salary, row_number() over(order by salary desc) rn from tab) t where rn=2 ").show()

1.M Write a query to find the second highest salary in an employee table.
+------+
|salary|
+------+
| 60000|
+------+



In [ ]:
print("2.M Fetch all employees whose names contain the letter “a” exactly twice.")
spark.sql("""
SELECT * FROM VALUES
(1, 'Ananya'),
(2, 'Aarav'),
(3, 'Aakash'),
(4, 'Aman'),
(5, 'Bala')
AS t(id, name);
""").createOrReplaceTempView("tab")
spark.sql("select * from tab where (length(lower(name)) - length(replace(lower(name),'a',''))) = 2 ").show()

2.M Fetch all employees whose names contain the letter “a” exactly twice.
+---+----+
| id|name|
+---+----+
|  4|Aman|
|  5|Bala|
+---+----+



In [ ]:
print("3.M How do you retrieve only duplicate records from a table?")
spark.sql('''SELECT * FROM VALUES
(1, 'Red', 100),
(2, 'Blue', 200),
(3, 'Red', 100),
(4, 'Green', 300)
AS t(id, color, value);
''').createOrReplaceTempView("tab")
spark.sql("select * from tab where id in (select count(1) from tab group by color,value having count(1)>1)").show()

3.M How do you retrieve only duplicate records from a table?
+---+-----+-----+
| id|color|value|
+---+-----+-----+
|  2| Blue|  200|
+---+-----+-----+



In [ ]:
print("4.M Write a query to calculate the running total of sales by date.")
spark.sql('''SELECT * FROM VALUES
('2024-06-01', 100),
('2024-06-02', 200),
('2024-06-03', 150),
('2024-06-04', 250),
('2024-06-05', 100),
('2024-06-06', 300)
AS t(sale_date, amount);''').createOrReplaceTempView("tab")
spark.sql("select sale_date, amount, sum(amount) over(order by sale_date rows between 2 preceding and current row) running_total from tab").show()

4.M Write a query to calculate the running total of sales by date.
+----------+------+-------------+
| sale_date|amount|running_total|
+----------+------+-------------+
|2024-06-01|   100|          100|
|2024-06-02|   200|          300|
|2024-06-03|   150|          450|
|2024-06-04|   250|          600|
|2024-06-05|   100|          500|
|2024-06-06|   300|          650|
+----------+------+-------------+



In [ ]:
print("5.M Find employees who earn more than the average salary in their department.")
spark.sql('''SELECT * FROM VALUES
(1, 'Alice', 50000, 1),
(2, 'Bob', 60000, 1),
(3, 'Charlie', 70000, 2),
(4, 'David', 40000, 2)
AS t(id, name, salary, department_id)''').createOrReplaceTempView("tab")
spark.sql("""
select e.* from tab e
join
(select department_id, avg(salary) avg_salary from tab group by department_id) d
on e.department_id = d.department_id
where e.salary > d.avg_salary
""").show()

5.M Find employees who earn more than the average salary in their department.
+---+-------+------+-------------+
| id|   name|salary|department_id|
+---+-------+------+-------------+
|  2|    Bob| 60000|            1|
|  3|Charlie| 70000|            2|
+---+-------+------+-------------+



In [ ]:
print("6.M Write a query to find the most frequently occurring value in a column.")
spark.sql('''
SELECT * FROM VALUES
('Apple'),
('Banana'),
('Apple'),
('Orange'),
('Apple')
AS t(fruit);''').createOrReplaceTempView("tab")
spark.sql("select fruit, count(1) cnt from tab group by fruit order by cnt desc limit 1").show()

6.M Write a query to find the most frequently occurring value in a column.
+-----+---+
|fruit|cnt|
+-----+---+
|Apple|  3|
+-----+---+



In [ ]:
print("7.M Fetch records where the date is within the last 7 days from today.")
spark.sql("""SELECT * FROM VALUES
('2025-03-27'),
('2025-05-27'),
('2025-06-01'),
('2025-06-03')
AS t(date_column);""").createOrReplaceTempView("tab")
spark.sql("select * from tab where date_column >= current_date - interval 7 day").show()

7.M Fetch records where the date is within the last 7 days from today.
+-----------+
|date_column|
+-----------+
| 2025-05-27|
| 2025-06-01|
| 2025-06-03|
+-----------+



In [2]:
# 8️⃣ How many share the same salary
spark.sql("""SELECT * FROM VALUES
(1, 'Alice', 50000),
(2, 'Bob', 60000),
(3, 'Charlie', 50000),
(4, 'David', 70000)
AS t(id, name, salary)""").createOrReplaceTempView("tab")
spark.sql("select salary, count(1) from tab group by salary having count(1)>1").show()

+------+--------+
|salary|count(1)|
+------+--------+
| 50000|       2|
+------+--------+



In [4]:
# 9️⃣ How do you fetch the top 3 records for each group in a table?
spark.sql("""SELECT * FROM VALUES
(1, 'A', 90),
(2, 'A', 80),
(3, 'A', 70),
(4, 'A', 60),
(5, 'B', 88),
(6, 'B', 77)
AS t(id, group_column, score);
""").createOrReplaceTempView("tab")
spark.sql("select * from (select *, row_number() over(partition by group_column order by score desc) rn from tab) t where rn<=3").show()

+---+------------+-----+---+
| id|group_column|score| rn|
+---+------------+-----+---+
|  1|           A|   90|  1|
|  2|           A|   80|  2|
|  3|           A|   70|  3|
|  5|           B|   88|  1|
|  6|           B|   77|  2|
+---+------------+-----+---+



In [5]:
#🔟 Products Never Sold (LEFT JOIN)
spark.sql("""
SELECT * FROM VALUES
(1, 'Laptop'),
(2, 'Mouse'),
(3, 'Keyboard')
AS products(product_id, name)""").createOrReplaceTempView("products")
spark.sql("""
SELECT * FROM VALUES
(1, 1000),
(2, 500)
AS sales(product_id, amount)""").createOrReplaceTempView("sales")

spark.sql("select * from products left join sales on products.product_id = sales.product_id where sales.product_id is null").show()

+----------+--------+----------+------+
|product_id|    name|product_id|amount|
+----------+--------+----------+------+
|         3|Keyboard|      NULL|  NULL|
+----------+--------+----------+------+



In [ ]:
# 1️⃣ Customers with First Purchase in Last 6 Months
spark.sql("""SELECT * FROM VALUES
(1, '2024-12-01'),
(2, '2025-02-15'),
(3, '2025-04-01'),
(4, '2023-11-20')
AS t(customer_id, purchase_date);
""").createOrReplaceTempView("tab")


In [ ]:
# 2️⃣ Pivot Table (Gender per Department
spark.sql("""SELECT * FROM VALUES
(1, 'M', 10),
(2, 'F', 10),
(3, 'M', 20),
(4, 'F', 20),
(5, 'M', 10)
AS t(emp_id, gender, department_id);
""").createOrReplaceTempView("tab")

In [ ]:
# 3️⃣ Month-over-Month Sales
spark.sql("""SELECT * FROM VALUES
('2025-01', 1000),
('2025-02', 1200),
('2025-03', 1000),
('2025-04', 1500)
AS t(sale_month, amount);
""").createOrReplaceTempView("tab")

In [ ]:
# 4️⃣ Median Salary
spark.sql("""SELECT * FROM VALUES
(1, 40000),
(2, 50000),
(3, 60000),
(4, 70000),
(5, 80000)
AS t(emp_id, salary);
""").createOrReplaceTempView("tab")

In [ ]:
# 5️⃣ Users with 3+ Consecutive Logins
spark.sql("""SELECT * FROM VALUES
(1, '2025-06-01'),
(1, '2025-06-02'),
(1, '2025-06-03'),
(2, '2025-06-01'),
(2, '2025-06-03')
AS t(user_id, login_date);
""").createOrReplaceTempView("tab")

In [ ]:
# 6️⃣ Delete Duplicate Rows, Keep One
spark.sql("""SELECT * FROM VALUES
(1, 'John', 50000),
(2, 'John', 50000),
(3, 'Alice', 60000)
AS t(id, name, salary);
""").createOrReplaceTempView("tab")

In [ ]:
# 7️⃣ Sales Ratio Between Categories
spark.sql("""SELECT * FROM VALUES
(1, 'A', 100),
(2, 'B', 50),
(3, 'A', 200),
(4, 'B', 150)
AS t(id, category, amount);
""").createOrReplaceTempView("tab")

In [ ]:
# 8️⃣ Recursive Hierarchy
spark.sql("""SELECT * FROM VALUES
(1, 'CEO', NULL),
(2, 'Manager1', 1),
(3, 'Manager2', 1),
(4, 'Dev1', 2),
(5, 'Dev2', 2)
AS t(id, name, manager_id);
""").createOrReplaceTempView("tab")

In [ ]:
# 9️⃣ Gaps in Sequential Numbers
spark.sql("""SELECT * FROM VALUES
(1), (2), (4), (5), (7)
AS t(current_number);
""").createOrReplaceTempView("tab")

In [ ]:
# 🔟 Split Comma-Separated String into Rows
spark.sql("""SELECT * FROM VALUES
(1, 'apple,banana,grape'),
(2, 'orange,mango')
AS t(id, names);
""").createOrReplaceTempView("tab")

In [ ]:
# 1️⃣ Rank Products by Sales in Region
spark.sql("""SELECT * FROM VALUES
('North', 'P1', 100),
('North', 'P2', 200),
('South', 'P1', 300),
('South', 'P3', 150)
AS t(region, product_id, sale_amount);
""").createOrReplaceTempView("tab")

In [ ]:
# 2️⃣ Top 10% Salaries per Department
spark.sql("""SELECT * FROM VALUES
(1, 'A', 50000),
(2, 'A', 55000),
(3, 'A', 60000),
(4, 'B', 70000),
(5, 'B', 80000)
AS t(id, department_id, salary);
""").createOrReplaceTempView("tab")

In [ ]:
# 3️⃣ Orders During Business Hours (9 AM - 6 PM)
spark.sql("""SELECT * FROM VALUES
(1, '2025-06-03 08:59:00'),
(2, '2025-06-03 09:30:00'),
(3, '2025-06-03 17:59:00'),
(4, '2025-06-03 18:01:00')
AS t(order_id, order_time);
""").createOrReplaceTempView("tab")

In [ ]:
# 4️⃣ Users Active on Weekdays and Weekends
spark.sql("""SELECT * FROM VALUES
(1, '2025-06-01'),  -- Sunday
(1, '2025-06-03'),  -- Tuesday
(2, '2025-06-02'),  -- Monday
(3, '2025-06-01'),  -- Sunday
(3, '2025-06-02')   -- Monday
AS t(user_id, activity_date);
""").createOrReplaceTempView("tab")

In [ ]:
# 5️⃣ Customers Purchasing Across 3+ Categories
spark.sql("""SELECT * FROM VALUES
(1, 'Electronics'),
(1, 'Clothing'),
(1, 'Home'),
(2, 'Clothing'),
(2, 'Clothing')
AS t(customer_id, category_id);
""").createOrReplaceTempView("tab")

In [ ]:
# 8️⃣ Recursive Hierarchy
spark.sql("""""").createOrReplaceTempView("tab")

In [ ]:
# 8️⃣ Recursive Hierarchy
spark.sql("""""").createOrReplaceTempView("tab")

In [ ]:
# 8️⃣ Recursive Hierarchy
spark.sql("""""").createOrReplaceTempView("tab")

In [ ]:
# 8️⃣ Recursive Hierarchy
spark.sql("""""").createOrReplaceTempView("tab")

In [ ]:
# 8️⃣ Recursive Hierarchy
spark.sql("""""").createOrReplaceTempView("tab")

In [ ]:
# 8️⃣ Recursive Hierarchy
spark.sql("""""").createOrReplaceTempView("tab")

In [ ]:
# 8️⃣ Recursive Hierarchy
spark.sql("""""").createOrReplaceTempView("tab")

In [ ]:
spark.sql("create table mytab as select * from values(1,'a'),(1,'a'),(1,'b'),(1,'b'),(1,'c'),(1,'d'),(1,'d') t(a,b)")

++
||
++
++



In [ ]:
spark.sql("select * from mytab").show()

+---+---+
|  a|  b|
+---+---+
|  1|  b|
|  1|  c|
|  1|  d|
|  1|  d|
|  1|  a|
|  1|  a|
|  1|  b|
+---+---+



In [ ]:
'''
+---+-------+--------+----------+--------+----------+
| id|   name|    city|start_date|end_date|is_current|
+---+-------+--------+----------+--------+----------+
|101|  Alice|New York|2023-01-01|    NULL|         Y|
|102|    Bob|  London|2023-01-05|    NULL|         Y|
|103|Charlie|   Paris|2023-01-10|    NULL|         Y|
+---+-------+--------+----------+--------+----------+

+---+-----+-----------+
| id| name|       city|
+---+-----+-----------+
|101|Alice|Los Angeles|
|102|  Bob|     London|
|104|David|     Berlin|
+---+-----+-----------+

+---+-------+--------+----------+--------+----------+
| id|   name|    city|start_date|end_date|is_current|
+---+-------+--------+----------+--------+----------+
|101|  Alice|New York|2023-01-01|2025-02-09|       N|
|101|  Alice|New York|2025-02-09|    NULL|         Y|
|102|    Bob|  London|2023-01-05|    NULL|         Y|
|103|Charlie|   Paris|2023-01-10|    NULL|         Y|
|104|David  |  Berlin|2025-02-09|    NULL|         Y|
+---+-------+--------+----------+--------+----------+

'''

In [ ]:
# Code for SCD Type 2
print('# --> Initial Target table: Historic table with customer information i.e target_df')
target_df = spark.sql('''
select * from values
 ('101','Alice','New York','2023-01-01',null, 'Y')
,('102','Bob','London','2023-01-05',null, 'Y')
,('103','Charlie','Paris','2023-01-10',null, 'Y')
t(id, name, city, start_date, end_date, is_current)
''')
target_df.show()
print('#--> incoming Souce Table : Latest snapshot from the source')
source_df = spark.sql('''
select * from values
('101','Alice','Los Angeles'),
('102','Bob','London'),
('104','David','Berlin')
t(id, name, city)
''')
source_df.show()

print('Step 1 --> Identify the changed records')
changed_df = target_df.alias("tgt").join(source_df.alias("src"),"id","left").filter(expr("tgt.name<>src.name or tgt.city <> src.city")).select("id")


print('# # Step 2 --> Mark old record ineactive and update end date')
target_df_delta = target_df.alias("tgt").join(changed_df.alias("chg"),"id","left")\
.withColumn("end_date",  expr("case when chg.id is not null then current_date() else tgt.end_date end"))\
.withColumn("is_current",expr("case when chg.id is not null then 'N' else tgt.is_current end"))
target_df_delta.show()

print('# Insert new and updated rewcords')
new_and_updated_df = source_df.alias("src").join(target_df.alias("trt"),"id","left").select("src.*")\
.withColumn("start_date",current_date())\
.withColumn("end_date",lit(None))\
.withColumn("is_active",lit('Y'))\
.union(target_df_delta)
new_and_updated_df.orderBy("id").show()

# --> Initial Target table: Historic table with customer information i.e target_df
+---+-------+--------+----------+--------+----------+
| id|   name|    city|start_date|end_date|is_current|
+---+-------+--------+----------+--------+----------+
|101|  Alice|New York|2023-01-01|    NULL|         Y|
|102|    Bob|  London|2023-01-05|    NULL|         Y|
|103|Charlie|   Paris|2023-01-10|    NULL|         Y|
+---+-------+--------+----------+--------+----------+

#--> incoming Souce Table : Latest snapshot from the source
+---+-----+-----------+
| id| name|       city|
+---+-----+-----------+
|101|Alice|Los Angeles|
|102|  Bob|     London|
|104|David|     Berlin|
+---+-----+-----------+

Step 1 --> Identify the changed records
# # Step 2 --> Mark old record ineactive and update end date
+---+-------+--------+----------+----------+----------+
| id|   name|    city|start_date|  end_date|is_current|
+---+-------+--------+----------+----------+----------+
|101|  Alice|New York|2023-01-01|2025-02

In [ ]:
target_df_delta = target_df.alias("tgt").join(changed_df.alias("chg"),"id","left")\
.withColumn("start_date",expr("case when chg.id is not null then current_date() else tgt.start_date end"))\
.withColumn("end_date",expr("case when chg.id is not null then Null else tgt.end_date end"))\
.withColumn("is_current",expr("case when chg.id is not null then 'N' else tgt.is_current end"))
target_df_delta.show()

+---+-------+--------+----------+--------+----------+
| id|   name|    city|start_date|end_date|is_current|
+---+-------+--------+----------+--------+----------+
|101|  Alice|New York|2025-02-10|    NULL|         N|
|102|    Bob|  London|2023-01-05|    NULL|         Y|
|103|Charlie|   Paris|2023-01-10|    NULL|         Y|
+---+-------+--------+----------+--------+----------+



In [ ]:
# Code for SCD Type 2
print('Initial Target table: Historic table with customer information i.e target_df')
target_df = spark.sql('''
select * from values
 ('101','Alice','New York','2023-01-01',null, 'Y')
,('102','Bob','London','2023-01-05',null, 'Y')
,('103','Charlie','Paris','2023-01-10',null, 'Y')
t(id, name, city, start_date, end_date, is_current)
''')
target_df.show()
print('Incoming Souce Table : Latest snapshot from the source')
source_df = spark.sql('''
select * from values
('101','Alice','Los Angeles'),
('102','Bob','London'),
('104','David','Berlin')
t(id, name, city)
''')
source_df.show()

# identify and udate existing rows
updated_df = target_df.alias("trg").join(source_df.alias("src"),"id","left")\
.filter("trg.name<>src.name or trg.city <> src.city").select("src.*")\
.withColumn("")
updated_df.show()
#


Initial Target table: Historic table with customer information i.e target_df
+---+-------+--------+----------+--------+----------+
| id|   name|    city|start_date|end_date|is_current|
+---+-------+--------+----------+--------+----------+
|101|  Alice|New York|2023-01-01|    NULL|         Y|
|102|    Bob|  London|2023-01-05|    NULL|         Y|
|103|Charlie|   Paris|2023-01-10|    NULL|         Y|
+---+-------+--------+----------+--------+----------+

Incoming Souce Table : Latest snapshot from the source
+---+-----+-----------+
| id| name|       city|
+---+-----+-----------+
|101|Alice|Los Angeles|
|102|  Bob|     London|
|104|David|     Berlin|
+---+-----+-----------+



TypeError: DataFrame.withColumn() missing 1 required positional argument: 'col'

In [ ]:
# Sample DataFrame creation (replace with your actual data loading)
data = [("1001", "Data Structures", 70),
        ("1001", "Java", 65),
        ("1001", "Software Engineering", 80),
        ("1001", "Maths", 76),
        ("1002", "Data Structures", 80),
        ("1002", "Java", 75),
        ("1002", "Software Engineering", 90),
        ("1002", "Maths", 86)]

columns = ["RollNo", "Subject", "Mark"]
student_df = spark.createDataFrame(data, columns)
student_df.createOrReplaceTempView("student")
#student_df.groupBy("RollNo").pivot("Subject", ["Data Structures", "Java", "Software Engineering", "Maths"]).sum("Mark").show()

spark.sql("""select *,Java +maths as s from (
  select rollNo, subject, mark from student
)
pivot(
  sum(mark) for subject in ('Data Structures', 'Java', 'Software Engineering', 'Maths')
)
""").show()


+------+---------------+----+--------------------+-----+---+
|rollNo|Data Structures|Java|Software Engineering|Maths|  s|
+------+---------------+----+--------------------+-----+---+
|  1002|             80|  75|                  90|   86|161|
|  1001|             70|  65|                  80|   76|141|
+------+---------------+----+--------------------+-----+---+



In [ ]:
#1. Write an SQL query to find the employees who are top 3 highest salary earners in each of the departments.
# Employee table:
# +----+-------+--------+--------------+
# | id | name  | salary | departmentId |
# +----+-------+--------+--------------+
# | 1  | Joe   | 85000  | 1            |
# | 2  | Henry | 80000  | 2            |
# | 3  | Sam   | 60000  | 2            |
# | 4  | Max   | 90000  | 1            |
# | 5  | Janet | 69000  | 1            |
# | 6  | Randy | 85000  | 1            |
# | 7  | Will  | 70000  | 1

# Department table:
# +----+-------+
# | id | name  |
# +----+-------+
# | 1  | IT    |
# | 2  | Sales |
# +----+-------+
# Output
# ---------
# empid, empname, empsalary, deptname
'''
SELECT id,name,salary,departmentName
FROM
    (SELECT e.id,e.name,e.salary,e.departmentId,d.name AS departmentName,
        DENSE_RANK() OVER (PARTITION BY e.departmentId ORDER BY e.salary DESC) as rank_within_department
    FROM Employee e JOIN Department d ON e.departmentId = d.id
     ) t
WHERE rank_within_department <= 3
ORDER BY departmentName, rank_within_department;
'''
''

''

In [ ]:
#2 prompt: write sql for below table to provide given output Sales table
# Store,custid,amt
# Input
spark.sql('''create or replace temp view tab as select * from values
(1,101,500),
(1,101,300),
(1,102,300),
(2,101,1000),
(2,102,900) as T(Store,custid,amt)''')
spark.sql("select * from tab").show()
# # Output: find sum of amt spend by 101 in each store
# # 1,101,800
# # 2,101,1000
spark.sql("""
select store, custid, sum(amt) from tab group by store, custid
""").filter("custid='101'").show()


+-----+------+----+
|Store|custid| amt|
+-----+------+----+
|    1|   101| 500|
|    1|   101| 300|
|    1|   102| 300|
|    2|   101|1000|
|    2|   102| 900|
+-----+------+----+

+-----+------+--------+
|store|custid|sum(amt)|
+-----+------+--------+
|    1|   101|     800|
|    2|   101|    1000|
+-----+------+--------+













3. You will be given 2 tables with some rows and join condition(left,inner,right). Qus. How many rows will be in the output for the respective join condition.

Python
1. Typical data structure in python( list, dict ) and operatoin on them? You will be asked to write code snipped for example slice the list. list comprehension etc.

In [ ]:
#3 SQL There are sales table, meterial table. when material is sold an entry is created in sales table. find the list of  material which have never been sold.

In [ ]:
# 4 What are the windows function?
'''
Aggregate Window Functions : SUM(), MAX(), MIN(), AVG(). COUNT()
Ranking Window Functions : RANK(), DENSE_RANK(), ROW_NUMBER(), NTILE()
Value Window Functions : LAG(), LEAD(), FIRST_VALUE(), LAST_VALUE()
'''
''

''

In [ ]:
data = [
(200100, 1000.00, 600.00, '2008-01-08', 'C00013', 'A003'),
(200112, 2000.00, 400.00, '2008-30-05', 'C00016', 'A007'),
(200113, 4000.00, 600.00, '2008-10-06', 'C00022', 'A002'),
(200122, 2500.00, 400.00, '2008-16-09', 'C00003', 'A004'),
(200118, 500.00, 100.00, '2008-20-07', 'C00023', 'A006'),
(200121, 1500.00, 600.00, '2008-23-09', 'C00008', 'A004'),
(200108, 4000.00, 600.00, '2008-15-02', 'C00008', 'A004'),
(200104, 1500.00, 500.00, '2008-13-03', 'C00006', 'A004'),
(200106, 2500.00, 700.00, '2008-20-04', 'C00005', 'A002'),
(200117, 800.00, 200.00, '2008-20-10', 'C00014', 'A001'),
(200123, 500.00, 100.00, '2008-16-09', 'C00022', 'A002'),
(200120, 500.00, 100.00, '2008-20-07', 'C00009', 'A002'),
(200124, 500.00, 100.00, '2008-20-06', 'C00017', 'A007'),
(200126, 500.00, 100.00, '2008-24-06', 'C00022', 'A002'),
(200129, 2500.00, 500.00, '2008-20-07', 'C00024', 'A006'),
(200127, 2500.00, 400.00, '2008-20-07', 'C00015', 'A003'),
(200128, 3500.00, 1500.00, '2008-20-07', 'C00009', 'A002'),
(200133, 1200.00, 400.00, '2008-29-06', 'C00009', 'A002')]

schema="ORD_NUM int,ORD_AMOUNT double,ADVANCE_AMOUNT double,ORD_DATE string,CUST_CODE string,AGENT_CODE string"

df = spark.createDataFrame(data,schema).withColumn("ORD_DATE",F.expr("to_date(ord_date,'yyyy-mm-dd')")).cache()
df.createOrReplaceTempView("orders")
spark.sql("select * from orders order by ORD_DATE, AGENT_CODE").show(50)

+-------+----------+--------------+----------+---------+----------+
|ORD_NUM|ORD_AMOUNT|ADVANCE_AMOUNT|  ORD_DATE|CUST_CODE|AGENT_CODE|
+-------+----------+--------------+----------+---------+----------+
| 200108|    4000.0|         600.0|2008-01-02|   C00008|      A004|
| 200104|    1500.0|         500.0|2008-01-03|   C00006|      A004|
| 200106|    2500.0|         700.0|2008-01-04|   C00005|      A002|
| 200112|    2000.0|         400.0|2008-01-05|   C00016|      A007|
| 200126|     500.0|         100.0|2008-01-06|   C00022|      A002|
| 200113|    4000.0|         600.0|2008-01-06|   C00022|      A002|
| 200133|    1200.0|         400.0|2008-01-06|   C00009|      A002|
| 200124|     500.0|         100.0|2008-01-06|   C00017|      A007|
| 200120|     500.0|         100.0|2008-01-07|   C00009|      A002|
| 200128|    3500.0|        1500.0|2008-01-07|   C00009|      A002|
| 200127|    2500.0|         400.0|2008-01-07|   C00015|      A003|
| 200118|     500.0|         100.0|2008-01-07|  

In [ ]:
# Qn 4.1. calculate the running total revenue for each agent in the third quarter:                 --and average revenue

spark.sql("""
select AGENT_CODE, ORD_AMOUNT, ORD_DATE,
  avg(ORD_AMOUNT) over(partition by AGENT_CODE order by ORD_DATE) tot_rev
from orders
""").show()

+----------+----------+----------+------------------+
|AGENT_CODE|ORD_AMOUNT|  ORD_DATE|           tot_rev|
+----------+----------+----------+------------------+
|      A001|     800.0|2008-01-10|             800.0|
|      A002|    2500.0|2008-01-04|            2500.0|
|      A002|    4000.0|2008-01-06|            2050.0|
|      A002|     500.0|2008-01-06|            2050.0|
|      A002|    1200.0|2008-01-06|            2050.0|
|      A002|     500.0|2008-01-07|2033.3333333333333|
|      A002|    3500.0|2008-01-07|2033.3333333333333|
|      A002|     500.0|2008-01-09|1814.2857142857142|
|      A003|    2500.0|2008-01-07|            2500.0|
|      A003|    1000.0|2008-01-08|            1750.0|
|      A004|    4000.0|2008-01-02|            4000.0|
|      A004|    1500.0|2008-01-03|            2750.0|
|      A004|    2500.0|2008-01-09|            2375.0|
|      A004|    1500.0|2008-01-09|            2375.0|
|      A006|     500.0|2008-01-07|            1500.0|
|      A006|    2500.0|2008-

In [ ]:
# Qn 4.2. How many days after the first purchase of a customer was the next purchase made?
spark.sql("""
select cust_code,ord_date, date_diff(ord_date , first_value(ord_date) over(partition by cust_code order by ord_date)) days from orders
""").orderBy("cust_code").show()

+---------+----------+----+
|cust_code|  ord_date|days|
+---------+----------+----+
|   C00003|2008-01-09|   0|
|   C00005|2008-01-04|   0|
|   C00006|2008-01-03|   0|
|   C00008|2008-01-02|   0|
|   C00008|2008-01-09|   7|
|   C00009|2008-01-06|   0|
|   C00009|2008-01-07|   1|
|   C00009|2008-01-07|   1|
|   C00013|2008-01-08|   0|
|   C00014|2008-01-10|   0|
|   C00015|2008-01-07|   0|
|   C00016|2008-01-05|   0|
|   C00017|2008-01-06|   0|
|   C00022|2008-01-06|   0|
|   C00022|2008-01-06|   0|
|   C00022|2008-01-09|   3|
|   C00023|2008-01-07|   0|
|   C00024|2008-01-07|   0|
+---------+----------+----+



In [ ]:
# Qn 3. what is the last highest amount for which an order was sold by an agent?
spark.sql("""
select agent_code, ORD_AMOUNT, ROW_NUMBER() OVER (PARTITION BY agent_code ORDER BY ord_amount DESC, ord_date DESC) as rn  from orders
""").show()


+----------+----------+---+
|agent_code|ORD_AMOUNT| rn|
+----------+----------+---+
|      A001|     800.0|  1|
|      A002|    4000.0|  1|
|      A002|    3500.0|  2|
|      A002|    2500.0|  3|
|      A002|    1200.0|  4|
|      A002|     500.0|  5|
|      A002|     500.0|  6|
|      A002|     500.0|  7|
|      A003|    2500.0|  1|
|      A003|    1000.0|  2|
|      A004|    4000.0|  1|
|      A004|    2500.0|  2|
|      A004|    1500.0|  3|
|      A004|    1500.0|  4|
|      A006|    2500.0|  1|
|      A006|     500.0|  2|
|      A007|    2000.0|  1|
|      A007|     500.0|  2|
+----------+----------+---+



In [ ]:
# Qn 4.4. what are the second highest order values for each month?
spark.sql("""SELECT * FROM (SELECT ord_num, ord_date, ord_amount, DENSE_RANK() OVER(PARTITION BY DATE_PART('month', ord_date)
        ORDER BY ord_amount DESC) order_rank
    FROM orders
) t
WHERE order_rank = 2
ORDER BY ord_date
""").show()

# spark.sql("""SELECT month(ord_date), agent_code, ord_amount, CUME_DIST() OVER(PARTITION BY month(ord_date) ORDER BY ord_amount) t from orders
# """).show()

+-------+----------+----------+----------+
|ord_num|  ord_date|ord_amount|order_rank|
+-------+----------+----------+----------+
| 200128|2008-01-07|    3500.0|         2|
+-------+----------+----------+----------+



In [ ]:
#5 write text to file and save as myfile.csv
# Find the count of each letter in the file
with open("/content/sample_data/myfile.csv","w") as f:
  f.write("""Hi Hello How are you Hi I am doing good
Hi Ram how are you doing
Hi Jack are you fine
Hello all
  """)
df = spark.read.csv("/content/sample_data/myfile.csv")
#df.withColumn("char",F.explode(F.split(F.lower(df._c0),""))).drop("_c0").groupBy("char").count().show()
df.createOrReplaceTempView("tab")
spark.sql("select t, count(1) from (select explode(split(_c0,'')) t from tab) a group by t  ").show()
# spark.read.csv("sample_data/california_housing_test.csv",header=True).show()

+---+--------+
|  t|count(1)|
+---+--------+
|  l|       6|
|  g|       3|
|  m|       2|
|  f|       1|
|  n|       3|
|  k|       1|
|  e|       6|
|  o|      11|
|  h|       1|
|  d|       3|
|  J|       1|
|  w|       2|
|  y|       3|
|  c|       1|
|  u|       3|
|  i|       7|
|  R|       1|
|  I|       1|
|  a|       7|
|  r|       3|
+---+--------+
only showing top 20 rows



In [ ]:
# 6
spark.sql("""
with Tab as (select * from values
(1,'y'),
(1,'y'),
(2,'n'),
(2,'n'),
(3,'y'),
(3,'n'),
(4,'y'),
(4,'y') t(ID,FLAG))
--expected id, YFlagCnt NFlagCnt
select id
, sum(case when flag='y' then 1 else 0 end) YFlagCnt
, sum(case when flag='n' then 1 else 0 end) NFlagCnt
from tab group by id
""").show()

# spark.sql('''
# select id, coalesce(y,0) as yCnt, coalesce(n,0) as Ncnt from
# (select id, flag from tab) t pivot (count(flag) for flag in ('y','n'))
# ''').show()

+---+--------+--------+
| id|YFlagCnt|NFlagCnt|
+---+--------+--------+
|  1|       2|       0|
|  2|       0|       2|
|  3|       1|       1|
|  4|       2|       0|
+---+--------+--------+



In [ ]:
spark.sql('''
with tab as (select * from values
 (1,'2021-10-28','home')
,(1,'2021-10-29','payment')
,(1,'2021-10-30','management')
,(1,'2021-10-28','home')
,(1,'2021-10-28','home') t
(userid, date, page_visited))

select date
, sum(case when page_visited='home' then 1 else 0 end) home_cnt
, sum(case when page_visited='payment' then 1 else 0 end) payment_cnt
, sum(case when page_visited='management' then 1 else 0 end) management_cnt
from tab group by date
''').show()

+----------+--------+-----------+--------------+
|      date|home_cnt|payment_cnt|management_cnt|
+----------+--------+-----------+--------------+
|2021-10-28|       3|          0|             0|
|2021-10-29|       0|          1|             0|
|2021-10-30|       0|          0|             1|
+----------+--------+-----------+--------------+



In [ ]:
a = [1,3,4]
b = a
b[1] = 100
print(a,b)

[1, 100, 4] [1, 100, 4]


In [ ]:
# Fine Column element in 2 list
a = [1,2,3,4,5]
b = [4,5,6,7,8]
def find_common(a,b):
  #return [x for x in a if x in b]
  return list(set(a) & set(b))
find_common(a,b)

[1, 2, 3, 6, 7, 8]

In [ ]:
from collections import Counter
Counter([1,2,2,2, 4, 3, 8, 9, 6, 6])[3]

1

In [ ]:
from collections import deque
d = deque([3,4,5])
d.append(6)
d.appendleft(2)
print(d)
d.pop()
d.popleft()
print(d)

deque([2, 3, 4, 5, 6])
deque([3, 4, 5])


In [ ]:
# Work Count Program
rdd = spark.sparkContext.textFile("/content/sample_data/myfile.csv")
lineRdd = rdd.flatMap(lambda line:line.split(" "))
words = lineRdd.map(lambda word:(word,1)).reduceByKey(lambda a,b:a+b)
words.collect()

[('How', 1),
 ('are', 3),
 ('you', 3),
 ('I', 1),
 ('doing', 2),
 ('good', 1),
 ('how', 1),
 ('Jack', 1),
 ('', 3),
 ('Hi', 4),
 ('Hello', 2),
 ('am', 1),
 ('Ram', 1),
 ('fine', 1),
 ('all', 1)]

In [ ]:
spark.sql("""
select * from values 1,2,3,4,5 as t(a) anti join values (3,2),(4,5),(7,5),(8,0),(9,4) b(a,c) on t.a=b.a
""").show()

+---+
|  a|
+---+
|  1|
|  2|
|  5|
+---+



In [ ]:
# Qns 5 out = ["ih","rogir"]
lst = ["hi", "rigor"]

lst = [ele[-1::-1] for ele in lst]
print(lst)

l = [1,1,1,2,3,3,4,4]

[(x,l.count(x)) for x in set(l)]

['ih', 'rogir']


[(1, 3), (2, 1), (3, 2), (4, 2)]

In [ ]:
spark.sql("""
select * from
values 1,1,2,2,null,2,4
join
values 1,2,2,3,3,null
""").count()

li = [1,[2,[3,[4]]]]
# out = li = [1,2,3,4]
flat_li = [item for sublist in li for item in (sublist if isinstance(sublist, list) else [sublist])]
flat_li

nlist = []
for item in li:
  if isinstance(item,list):
    nlist.extend(item)
  else:
    nlist.append(item)
print(nlist)


[1, 2, [3, [4]]]


In [ ]:
# Rename multiple column in spark in one statement
spark.sql("select null as a, null as b, null c").toDF("x","y","z").show()

+----+----+----+
|   x|   y|   z|
+----+----+----+
|NULL|NULL|NULL|
+----+----+----+



In [ ]:
spark.sql("""
with mytemp_cte
(
  select * from values 10000,10000,3000,5000 T(sal)
),
mytemp1_cte
(
  select 1 as t
)
select sal, sal - avg(sal) over(partition by 1) diff from mytemp_cte join mytemp1_cte order by sal desc limit 10
""").show()

+-----+-------+
|  sal|   diff|
+-----+-------+
|10000| 3000.0|
|10000| 3000.0|
| 5000|-2000.0|
| 3000|-4000.0|
+-----+-------+



In [ ]:
# Qn . 7 SQL  to fetch the firt name , emp_no  and the count of teh no of  times their salary change over the year whose employee no is 10001
# Note :- The result should be grouped first_name and e.emp_no
# Emp Table
# Emp_no, first_name, gender
# Salary table
#  emp_No, salary , from_date, to_date
# output:
# First_nbame Emp_no, Count
#  Sams 1001 2

In [ ]:
# spark.sql("""
# select * from values (10001,'Sam','M') t(emp_no,sal,gender)
# """).show()

spark.sql("""
with emptab (select * from values
(10001,10000,to_date('2021-11-04'),to_date('2021-11-09')),
(10001,12000,to_date('2022-11-09'),to_date('2022-03-09')),
(10001,14000,to_date('2022-04-09'),to_date('2022-05-09')),
(10001,16000,to_date('2022-05-04'),to_date('2022-07-09')),
(10001,18000,to_date('2022-07-04'),to_date('2022-11-09'))
t(emp_no,sal,from_date,to_date))

select emp_no,year(from_date), count(*) from emptab where year(from_date) = year(current_date) - 1 group by emp_no, year(from_date)

""").show()

+------+---------------+--------+
|emp_no|year(from_date)|count(1)|
+------+---------------+--------+
| 10001|           2022|       4|
+------+---------------+--------+



In [ ]:
spark.sql("""
with student(
  select * from values
('a','math',44),
('a','eng',54),
('a','phy',74),
('b','math',44),
('b','eng',54),
('b','phy',74)
T(name,sub,marks))

select * from (

select name,sub,marks from student

)
pivot
(
  max(marks) for sub in ('math','phy','eng')
)
""").show()

+----+----+---+---+
|name|math|phy|eng|
+----+----+---+---+
|   b|  44| 74| 54|
|   a|  44| 74| 54|
+----+----+---+---+



Description : You are given a time series data, which is a clickstream of user activity. Perform Sessionization

on the data {as per the session definition given below} and generate session ids.

Expected Steps:
1. Create in an input file with the data given below in any flat file format, preferably csv.
2. Read the input data file into your program
3. Use spark batch {PySpark/Spark-Scala} to add an additional column with name session_id and generate the session
id's based on the following logic:
    Session expires after inactivity of 30 minutes; because of inactivity, no clickstream will be generated.
    Session remains active for a maximum duration of 2 hours (i.e., after every two hours a new session starts).
4. Save the resultant data (original data, enriched with Session IDs) in a Parquet file format.   
Given Data

```
# This is formatted as code
  Timestamp             User_id
  2021-05-01T11:00:00Z      u1
  2021-05-01T13:13:00Z      u1
  2021-05-01T15:00:00Z      u2
  2021-05-01T11:25:00Z      u1
  2021-05-01T15:15:00Z      u2
  2021-05-01T02:13:00Z      u3
  2021-05-03T02:15:00Z      u4
  2021-05-02T11:45:00Z      u1
  2021-05-02T11:00:00Z      u3
  2021-05-03T12:15:00Z      u3
  2021-05-03T11:00:00Z      u4
  2021-05-03T21:00:00Z      u4
  2021-05-04T19:00:00Z      u2
  2021-05-04T09:00:00Z      u3
  2021-05-04T08:15:00Z      u1
```
Expected Output

  
```
TimeStamp             User_id  
<b>Session_id(<userid>_<session_number>)</b>
2021-05-01T11:00:00Z    u1     For example, u1_s1
2021-05-01T13:13:00Z    u1     For example, u1_s2
2021-05-01T15:00:00Z    u2     For example, u2_s1
2021-05-01T11:25:00Z    u1     Please derive based on given logic
2021-05-01T15:15:00Z    u2     Please derive based on given logic
2021-05-01T02:13:00Z    u3     Please derive based on given logic
2021-05-03T02:15:00Z    u4     Please derive based on given logic
2021-05-02T11:45:00Z    u1     Please derive based on given logic
2021-05-02T11:00:00Z    u3     Please derive based on given logic
2021-05-03T12:15:00Z    u3     Please derive based on given logic
2021-05-03T11:00:00Z    u4     Please derive based on given logic
2021-05-03T21:00:00Z    u4     Please derive based on given logic
2021-05-04T19:00:00Z    u2     Please derive based on given logic
2021-05-04T09:00:00Z    u3     Please derive based on given logic
2021-05-04T08:15:00Z    u1     Please derive based on given logic

```




In [ ]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.sql.types import StringType



In [ ]:
df1 = spark.sql('''
select cast(Timestamp as timestamp) Timestamp,User_Id from values
('2021-05-01T11:00:00Z','u1'),
('2021-05-01T13:13:00Z','u1'),
('2021-05-01T15:00:00Z','u2'),
('2021-05-01T11:25:00Z','u1'),
('2021-05-01T15:15:00Z','u2'),
('2021-05-01T02:13:00Z','u3'),
('2021-05-03T02:15:00Z','u4'),
('2021-05-02T11:45:00Z','u1'),
('2021-05-02T11:00:00Z','u3'),
('2021-05-03T12:15:00Z','u3'),
('2021-05-03T11:00:00Z','u4'),
('2021-05-03T21:00:00Z','u4'),
('2021-05-04T19:00:00Z','u2'),
('2021-05-04T09:00:00Z','u3'),
('2021-05-04T08:15:00Z','u1')
t(Timestamp,User_Id)
''').cache()


In [ ]:
#
df1.orderBy("User_id","Timestamp")\
.withColumn("ltd",F.expr("unix_timestamp(Timestamp) - unix_timestamp(lag(Timestamp) over(partition by user_id order by Timestamp))")/60)\
.withColumn("session_id",F.expr("sum(case when ltd > 30 then 1 else 0 end) over(partition by user_id order by Timestamp) + sum(case when ltd > 120 then 1 else 0 end) over(partition by user_id order by Timestamp)"))\
.show(1000,0)

+-------------------+-------+------+----------+
|Timestamp          |User_Id|ltd   |session_id|
+-------------------+-------+------+----------+
|2021-05-01 11:00:00|u1     |NULL  |0         |
|2021-05-01 11:25:00|u1     |25.0  |0         |
|2021-05-01 13:13:00|u1     |108.0 |1         |
|2021-05-02 11:45:00|u1     |1352.0|3         |
|2021-05-04 08:15:00|u1     |2670.0|5         |
|2021-05-01 15:00:00|u2     |NULL  |0         |
|2021-05-01 15:15:00|u2     |15.0  |0         |
|2021-05-04 19:00:00|u2     |4545.0|2         |
|2021-05-01 02:13:00|u3     |NULL  |0         |
|2021-05-02 11:00:00|u3     |1967.0|2         |
|2021-05-03 12:15:00|u3     |1515.0|4         |
|2021-05-04 09:00:00|u3     |1245.0|6         |
|2021-05-03 02:15:00|u4     |NULL  |0         |
|2021-05-03 11:00:00|u4     |525.0 |2         |
|2021-05-03 21:00:00|u4     |600.0 |4         |
+-------------------+-------+------+----------+



In [ ]:
sess_w = Window.partitionBy('User_id').orderBy("TimeStamp")

time_diff_minutes = (F.unix_timestamp("Timestamp") - F.unix_timestamp(F.lag("Timestamp").over(sess_w))) / 60

In [ ]:
sw = Window.partitionBy("User_id").orderBy("TimeStamp")
td = (F.unix_timestamp(F.col("timestamp")) - F.unix_timestamp(F.lag("timestamp").over(sw)) ) / 60

In [ ]:
session_boundary = (sum(when(time_diff_minutes >= 30, 1).otherwise(0)).over(session_window) +
                    when(time_diff_minutes >= 120, 1).otherwise(0)).cast("string")


In [ ]:
scol = (sum(F.when(td >= 30,1).otherwise(0).over(sw) + F.when(td >= 120,1).otherwise(0) )).cast(StringType())

PySparkTypeError: [NOT_ITERABLE] Column is not iterable.

In [ ]:
df1.withColumn("session_id", scol ).show()

AnalysisException: It is not allowed to use a window function inside an aggregate function. Please use the inner window function in a sub-query.

In [ ]:
# from chatGPT
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, unix_timestamp, lag, when, sum
from pyspark.sql.window import Window

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Sessionization") \
    .getOrCreate()

# Load the data
data = [("2021-05-01T11:00:00Z", "u1"),
        ("2021-05-01T13:13:00Z", "u1"),
        ("2021-05-01T15:00:00Z", "u2"),
        ("2021-05-01T11:25:00Z", "u1"),
        ("2021-05-01T15:15:00Z", "u2"),
        ("2021-05-01T02:13:00Z", "u3"),
        ("2021-05-03T02:15:00Z", "u4"),
        ("2021-05-02T11:45:00Z", "u1"),
        ("2021-05-02T11:00:00Z", "u3"),
        ("2021-05-03T12:15:00Z", "u3"),
        ("2021-05-03T11:00:00Z", "u4"),
        ("2021-05-03T21:00:00Z", "u4"),
        ("2021-05-04T19:00:00Z", "u2"),
        ("2021-05-04T09:00:00Z", "u3"),
        ("2021-05-04T08:15:00Z", "u1")]

columns = ["timestamp", "user_id"]
df = spark.createDataFrame(data, columns)

# Convert timestamp to TimestampType
df = df.withColumn("timestamp", col("timestamp").cast("timestamp"))

# Define Window
window_spec = Window.partitionBy("user_id").orderBy("timestamp")

# Calculate time difference in seconds from the previous event
df = df.withColumn("prev_timestamp", lag("timestamp").over(window_spec)) \
    .withColumn("time_diff", (unix_timestamp("timestamp") - unix_timestamp("prev_timestamp")) / 60)

# Determine session break (30 minutes or 2 hours max session duration)
df = df.withColumn("new_session", when((col("time_diff") > 30) | (col("time_diff").isNull()), 1).otherwise(0))

# Assign session IDs
df = df.withColumn("session_number", sum("new_session").over(window_spec.rowsBetween(Window.unboundedPreceding, 0))) \
    .withColumn("session_id", col("user_id") + "_s" + col("session_number").cast("string"))

# Save result to Parquet
df.show()


+-------------------+-------+-------------------+---------+-----------+--------------+----------+
|          timestamp|user_id|     prev_timestamp|time_diff|new_session|session_number|session_id|
+-------------------+-------+-------------------+---------+-----------+--------------+----------+
|2021-05-01 11:00:00|     u1|               NULL|     NULL|          1|             1|      NULL|
|2021-05-01 11:25:00|     u1|2021-05-01 11:00:00|     25.0|          0|             1|      NULL|
|2021-05-01 13:13:00|     u1|2021-05-01 11:25:00|    108.0|          1|             2|      NULL|
|2021-05-02 11:45:00|     u1|2021-05-01 13:13:00|   1352.0|          1|             3|      NULL|
|2021-05-04 08:15:00|     u1|2021-05-02 11:45:00|   2670.0|          1|             4|      NULL|
|2021-05-01 15:00:00|     u2|               NULL|     NULL|          1|             1|      NULL|
|2021-05-01 15:15:00|     u2|2021-05-01 15:00:00|     15.0|          0|             1|      NULL|
|2021-05-04 19:00:00

In [ ]:
Tab1
tabA   tabB
id     id
1		2
2		4
3		Null
4		2
5		4
Null	Null
Select Count(*), Count(ColA), count(ColB) from Tab1;

left join -> 5  (9)
inner join -> 4 (6)
right join -> 4 (6)
full outer join -> 6

In [ ]:
d1 = spark.sql("""
select * from values
(1),
(2),
(3),
(4),
(5),
(Null) as T(id)""")
d2 = spark.sql("""select * from values
(2),
(4),
(Null),
(2),
(4),
(Null) as T(id)""")

d1.join(d2,d1.id==d2.id,"left").show()

+----+----+
|  id|  id|
+----+----+
|   1|NULL|
|   2|   2|
|   2|   2|
|   3|NULL|
|   4|   4|
|   4|   4|
|   5|NULL|
|NULL|NULL|
+----+----+



In [ ]:
Tab1
tabA   tabB
id     id
1		   2
2		   4
3		   Null
4		   2
5		   4
Null	 Null
Select Count(*), Count(ColA), count(ColB) from Tab1;

left:16 (8)
right: 13 (6)
inner: 16 (5)
full:  16 (8)

CustomerTab, Order_Tab

id, name     order_id, order_date

In [ ]:
# from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.sql.types import StringType

# Create a Spark session
# spark = SparkSession.builder.appName("Sessionization").getOrCreate()

# Read the input CSV file
# df = spark.read.csv("clickstream_data.csv", header=True)

# Convert the timestamp column to a timestamp type
df = df.withColumn("Timestamp", F.to_timestamp(df["Timestamp"]))

# Define sessionization logic
session_window = Window.partitionBy("User_id").orderBy("Timestamp")

# Calculate time difference in minutes from the previous row
time_diff_minutes = (F.unix_timestamp("Timestamp") - F.unix_timestamp(F.lag("Timestamp").over(session_window))) / 60


# Generate session IDs
df = df.withColumn("Session_id",
F.concat_ws("_", df["User_id"],

  (F.sum(F.when(time_diff_minutes >= 30, 1).otherwise(0)).over(session_window) + F.when(time_diff_minutes >= 120, 1).otherwise(0)).cast(StringType())))

df.show()
# # Save the result in Parquet format
# df.write.parquet("sessionized_data.parquet")

# # Stop the Spark session
# spark.stop()


+-------------------+-------+----------+
|          Timestamp|User_Id|Session_id|
+-------------------+-------+----------+
|2021-05-01 11:00:00|     u1|      u1_0|
|2021-05-01 11:25:00|     u1|      u1_0|
|2021-05-01 13:13:00|     u1|      u1_1|
|2021-05-02 11:45:00|     u1|      u1_3|
|2021-05-04 08:15:00|     u1|      u1_4|
|2021-05-01 15:00:00|     u2|      u2_0|
|2021-05-01 15:15:00|     u2|      u2_0|
|2021-05-04 19:00:00|     u2|      u2_2|
|2021-05-01 02:13:00|     u3|      u3_0|
|2021-05-02 11:00:00|     u3|      u3_2|
|2021-05-03 12:15:00|     u3|      u3_3|
|2021-05-04 09:00:00|     u3|      u3_4|
|2021-05-03 02:15:00|     u4|      u4_0|
|2021-05-03 11:00:00|     u4|      u4_2|
|2021-05-03 21:00:00|     u4|      u4_3|
+-------------------+-------+----------+

